In [15]:
!pip install -q kagglehub tensorflow matplotlib scikit-learn


In [31]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet50 import preprocess_input
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score
import kagglehub


In [17]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print("✅ GPU detected")
else:
    print("⚠️ GPU not found")


✅ GPU detected


In [18]:
dataset_path = kagglehub.dataset_download("sumn2u/garbage-classification-v2")
data_dir = dataset_path
print("Dataset path:", data_dir)


Using Colab cache for faster access to the 'garbage-classification-v2' dataset.
Dataset path: /kaggle/input/garbage-classification-v2


In [19]:
from PIL import Image

def clean_dataset(root_dir):
    valid_ext = (".jpg", ".jpeg", ".png", ".bmp")
    removed = 0

    for root, _, files in os.walk(root_dir):
        for file in files:
            path = os.path.join(root, file)

            if not file.lower().endswith(valid_ext):
                os.remove(path)
                removed += 1
                continue

            try:
                img = Image.open(path)
                img.verify()
            except:
                os.remove(path)
                removed += 1

    print(f"Removed {removed} corrupted / invalid files")

clean_dataset(data_dir)


Removed 0 corrupted / invalid files


In [20]:
SEED = 42
IMG_SIZE = (256, 256)
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

temp_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
NUM_CLASSES = len(class_names)

temp_batches = tf.data.experimental.cardinality(temp_ds).numpy()
val_ds = temp_ds.take(temp_batches // 2)
test_ds = temp_ds.skip(temp_batches // 2)

print("Classes:", class_names)
print("Train batches:", tf.data.experimental.cardinality(train_ds).numpy())
print("Val batches:", tf.data.experimental.cardinality(val_ds).numpy())
print("Test batches:", tf.data.experimental.cardinality(test_ds).numpy())


Found 20212 files belonging to 10 classes.
Using 16170 files for training.
Found 20212 files belonging to 10 classes.
Using 4042 files for validation.
Classes: ['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']
Train batches: 506
Val batches: 63
Test batches: 64


In [21]:
def preprocess(image, label):
    image = preprocess_input(image)
    return image, label

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_ds   = val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_ds  = test_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

train_ds = train_ds.ignore_errors().prefetch(tf.data.AUTOTUNE)
val_ds   = val_ds.ignore_errors().prefetch(tf.data.AUTOTUNE)
test_ds  = test_ds.ignore_errors().prefetch(tf.data.AUTOTUNE)



In [22]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])


In [23]:
base_model = keras.applications.ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(256, 256, 3)
)

base_model.trainable = False

inputs = keras.Input(shape=(256, 256, 3))
x = data_augmentation(inputs)
x = base_model(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs, outputs)


In [28]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=5e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[
        "accuracy"
    ]
)

model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 8, 8, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │        20,490 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,608,202 (90.06 MB)

 Trainable params: 20,490 (80.04 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [29]:
EPOCHS = 8

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)


Epoch 1/8
504/504 ━━━━━━━━━━━━━━━━━━━━ 110s 202ms/step - accuracy: 0.2915 - loss: 2.3758 - val_accuracy: 0.6845 - val_loss: 0.9973
Epoch 2/8
504/504 ━━━━━━━━━━━━━━━━━━━━ 142s 281ms/step - accuracy: 0.6274 - loss: 1.1284 - val_accuracy: 0.7956 - val_loss: 0.6409
Epoch 3/8
504/504 ━━━━━━━━━━━━━━━━━━━━ 100s 198ms/step - accuracy: 0.7376 - loss: 0.8029 - val_accuracy: 0.8472 - val_loss: 0.4968
Epoch 4/8
504/504 ━━━━━━━━━━━━━━━━━━━━ 99s 197ms/step - accuracy: 0.7855 - loss: 0.6574 - val_accuracy: 0.8656 - val_loss: 0.4192
Epoch 5/8
504/504 ━━━━━━━━━━━━━━━━━━━━ 100s 197ms/step - accuracy: 0.8139 - loss: 0.5723 - val_accuracy: 0.8760 - val_loss: 0.3909
Epoch 6/8
504/504 ━━━━━━━━━━━━━━━━━━━━ 100s 198ms/step - accuracy: 0.8327 - loss: 0.5051 - val_accuracy: 0.8849 - val_loss: 0.3506
Epoch 7/8
504/504 ━━━━━━━━━━━━━━━━━━━━ 100s 198ms/step - accuracy: 0.8433 - loss: 0.4664 - val_accuracy: 0.8943 - val_loss: 0.3244
Epoch 8/8
504/504 ━━━━━━━━━━━━━━━━━━━━ 150s 297ms/step - accuracy: 0.8572 - loss: 0.

In [38]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"\nAccuracy: {test_acc*100:.2f}%")



63/63 ━━━━━━━━━━━━━━━━━━━━ 15s 176ms/step - accuracy: 0.9093 - loss: 0.2975

Accuracy: 91.12%


In [39]:
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    preds = preds.argmax(axis=1)

    y_true.extend(labels.numpy())
    y_pred.extend(preds)

precision = precision_score(
    y_true, y_pred, average="weighted"
)
recall = recall_score(
    y_true, y_pred, average="weighted"
)

cm = confusion_matrix(y_true, y_pred)
print(cm)


print("Precision:", f"{precision*100:.2f}%")
print("Recall:", f"{recall*100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [40]:
model.save("model.keras")